In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd

In [2]:
def find_elements_by_attribute(root, attribute_name, attribute_value):
    return root.findall(".//*[@{}='{}']".format(attribute_name, attribute_value))

In [3]:
def list_all_folders(directory):
    folders = []
    for item in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, item)):
            folders.append(item)
    return folders

In [173]:
def wrong_correct_sentence(root,begin,end,correct_word):
    wrong_sentence , correct_sentence="",""
    for element in root.findall("{http:///de/tudarmstadt/ukp/dkpro/core/api/segmentation/type.ecore}Sentence"):
        if int(element.get("begin"))<= begin and int(element.get("end"))>=end:
            wrong_sentence=root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[int(element.get("begin")):int(element.get("end"))] 
            s_begin=int(element.get("begin"))
            s_end=int(element.get("end"))
            if correct_word==None:
                correct_word=" "
            correct_sentence=root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[s_begin:begin]+correct_word+root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[end:s_end] 

    return wrong_sentence, correct_sentence


In [174]:
data={
    'file_name':[],
    'USER_NUMBER':[],
    'begin':[],
    'end':[],
    'wrong_word':[],
    'correct_word':[],
    'wrong_sentence':[],
    'correct_sentence':[],
    'process':[],
    'main_category':[],
    'sub_category_1':[],
    'sub_category_2':[]

}
df = pd.DataFrame(data)
df.to_csv('data.csv')

In [175]:
def extract_data_from_XMI(path,USER_NUMBER):
    data_temp={
    'file_name':[],
    'USER_NUMBER':[],
    'begin':[],
    'end':[],
    'wrong_word':[],
    'correct_word':[],
    'wrong_sentence':[],
    'correct_sentence':[],
    'process':[],
    'main_category':[],
    'sub_category_1':[],
    'sub_category_2':[]
}
    tree = ET.parse(path)
    root=tree.getroot()
    if len(root.findall('{http:///webanno/custom.ecore}Orthography'))!=0 :
        # print(root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString"))
        for element in root.findall('{http:///webanno/custom.ecore}Orthography'):
            data_temp['USER_NUMBER'].append(USER_NUMBER)
            begin=element.get("begin")
            end=element.get("end")
            data_temp['file_name'].append(root.findall("{http:///de/tudarmstadt/ukp/dkpro/core/api/metadata/type.ecore}DocumentMetaData")[0].get("documentTitle"))
            data_temp['begin'].append(begin)
            data_temp['end'].append(end)
            data_temp['correct_word'].append(element.get("correction"))
            data_temp['main_category'].append(element.get("mainCategory"))
            data_temp['sub_category_1'].append(element.get("subCategory_1"))
            data_temp['wrong_word'].append(root.findall("{http:///uima/cas.ecore}Sofa")[0].get("sofaString")[int(begin):int(end)])
            data_temp['process'].append(element.get("process"))
            if "subCategory_2" in element.attrib:
                data_temp["sub_category_2"].append(element.get("subCategory_2"))
            else:
                data_temp["sub_category_2"].append("None")
            wrong_sentence,correct_sentence=wrong_correct_sentence(root,int(begin),int(end),element.get("correction"))
            data_temp["wrong_sentence"].append(wrong_sentence)
            data_temp["correct_sentence"].append(correct_sentence)

    df2=pd.DataFrame(data_temp)
    return df2

    

In [176]:
annotation_folders=list_all_folders("./all_data")
for folder in annotation_folders:
    all_items = os.listdir("./all_data/"+folder)
    xmi_files = [f for f in all_items if f.endswith('.xmi') and os.path.isfile(os.path.join("./all_data",folder, f))]
    for xmi_file in xmi_files:
        xmi_path=os.path.join("./all_data",folder,xmi_file)
    output= extract_data_from_XMI(xmi_path,xmi_file[:-4])
    if len(output)!=0:
        df=pd.concat([df,output],axis=0)
    


In [177]:
df.to_csv('specll_correction.csv')